# model.ipynb


## Teilauftrag 3: Vorhersagemodell

**Autor**: Linhard Zejneli  
**Datum**: 3.04.2025

In diesem Notebook erstelle und validiere ich ein Vorhersagemodell für das **OVR**-Feld aus meinem EA Sports FC 25 Datensatz. Dazu splitte ich die Daten in Trainings- und Test-Sets, wähle einen geeigneten Algorithmus, trainiere ihn und bewerte die Ergebnisse mithilfe von Fehlermassen (MSE, RMSE, R²). Abschliessend überprüfe ich anhand einzelner Beispieldaten, ob die Vorhersagen **plausibel** sind.


## 1) Vorbereitung und Datensatz

Ich nutze erneut dieselben Daten wie in Teilauftrag 2, wobei der Datensatz bereits bereinigt und in `data/cleaned_fc25_players_bereinigt.xlsx` gespeichert ist. Dort sind numerische Felder wie PAC, SHO, PAS, DRI, DEF und PHY enthalten. Sie dienen als Eingabe (Features), um das Ziel-Feld **OVR** vorherzusagen. Zunächst lade ich die Daten und mache eine Train-/Test-Aufteilung, um das Modell auf unbekannten Daten prüfen zu können.


In [6]:
import pandas as pd
import numpy as np


df = pd.read_excel("data/cleaned_fc25_players_bereinigt.xlsx")

# Eingabedaten (Features)
features = ["PAC","SHO","PAS","DRI","DEF","PHY"]
# Zielvariable
target = "OVR"

X = df[features]
y = df[target]

print("Daten geladen. Dimensionen:")
print("X-Shape:", X.shape)
print("y-Shape:", y.shape)


Daten geladen. Dimensionen:
X-Shape: (17737, 6)
y-Shape: (17737,)


## 2) Train-/Test-Split

Um Overfitting zu vermeiden, teile ich meine Daten in 80% Training und 20% Test. So kann ich später objektiv prüfen, wie gut das Modell generalisiert, anstatt nur die Leistung auf den Trainingsdaten zu betrachten.


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Trainingsmenge:", X_train.shape)
print("Testmenge:", X_test.shape)


Trainingsmenge: (14189, 6)
Testmenge: (3548, 6)


## 3) Algorithmuswahl 

Ich entscheide mich für den **RandomForestRegressor**, da er sich hervorragend für tabellarische Daten eignet und nicht nur lineare Zusammenhänge abbilden kann. Im Gegensatz zu einem einfachen linearen Modell berücksichtigt er nichtlineare Effekte zwischen PAC, SHO, PAS, DRI, DEF, PHY und dem OVR. Random Forests neigen ausserdem weniger zum Overfitting als ein reiner Entscheidungsbaum und können dank Bagging robust gegenüber Rauschen oder Ausreissern sein. Darüber hinaus geben sie Einblick in die Feature-Bedeutungen, was aufzeigt, welche Attribute das OVR am stärksten beeinflussen. Diese Eigenschaften machen den RandomForestRegressor zu einer exzellenten Wahl für mein Projekt.


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Modell initialisieren
model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,  # für Reproduzierbarkeit
)

# Training
model.fit(X_train, y_train)

# Vorhersage
y_pred = model.predict(X_test)

# Auswertung
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("=== Ergebnis Metriken ===")
print(f"MSE  : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.3f}")


=== Ergebnis Metriken ===
MSE  : 1.58
RMSE : 1.26
R²   : 0.967


## 4) Plausibilitätsprüfung 

Neben den statistischen Metriken wie RMSE oder R² überprüfe ich das Modell anhand einiger echter Testbeispiele, um zu sehen, ob die Vorhersagen sinnvoll erscheinen. Wenn ein Spieler beispielsweise sehr hohe Werte bei PAC und SHO, aber eine schwache Defense hat, erwarte ich ein überdurchschnittlich hohes OVR. Liegt das Modell in einem Bereich von ±3 bis ±5 Punkten daneben, ist das für ein Rating-System akzeptabel. Sollten sich jedoch extreme Abweichungen zeigen (z. B. 20 Punkte Unterschied), wäre das ein Signal, dass wir entweder mehr Daten, andere Features oder bessere Hyperparameter brauchen.


In [10]:
# Manuelle Prüfung an einigen Beispieldaten
sample_indices = X_test.iloc[:5].index  # z.B. die ersten 5 Zeilen aus X_test
predictions = model.predict(X_test.iloc[:5])

comparison_df = pd.DataFrame({
    "PAC": X_test.loc[sample_indices, "PAC"].values,
    "SHO": X_test.loc[sample_indices, "SHO"].values,
    "DEF": X_test.loc[sample_indices, "DEF"].values,
    "Real OVR": y_test.loc[sample_indices].values,
    "Predicted OVR": predictions
}, index=sample_indices)

print("=== Manuelle Prüfung (erste 5 Testinstanzen) ===")
display(comparison_df)


=== Manuelle Prüfung (erste 5 Testinstanzen) ===


,PAC,SHO,DEF,Real OVR,Predicted OVR
14352,61,29,56,58,56.07
7648,77,65,58,66,66.67
12245,68,63,33,62,62.49
4315,55,52,70,70,69.76
13425,78,50,53,60,60.53


## Warum ich die bisherige Prüfung nicht sinnvoll finde und eine bessere wähle

Meiner Ansicht nach ist die bisherige Testmethode (rein zufällige Auswahl von Spielern) nur bedingt aussagekräftig. Zwar erhält man so zufällige Beispiele aus dem Datensatz, doch spiegelt das nicht zwangsläufig die fußballtypischen Profile wider, wie man sie in der Praxis sehen würde. Deshalb konzentriere ich mich lieber auf gezielte Spielertypen, beispielsweise sehr starke Angreifer (hohe PAC, SHO) oder Verteidiger (hohe DEF, niedriges SHO). Auf diese Weise lässt sich viel klarer erkennen, ob das Modell tatsächlich versteht, was einen guten Stürmer oder Verteidiger auszeichnet.


In [11]:
import numpy as np
import pandas as pd


# 1) Angreifer-Szenario: Hohe PAC & SHO
#    Hier definieren wir "hoch" mal als >80 für beides
attack_mask = (X_test["PAC"] > 80) & (X_test["SHO"] > 80)
attack_indices = X_test[attack_mask].index

if len(attack_indices) == 0:
    print("Keine Angreifer mit (PAC>80 & SHO>80) im Testset!")
else:
   
    if len(attack_indices) > 5:
        attack_indices = np.random.choice(attack_indices, 5, replace=False)

    preds_attack = model.predict(X_test.loc[attack_indices])
    df_attack = pd.DataFrame({
        "PAC": X_test.loc[attack_indices, "PAC"],
        "SHO": X_test.loc[attack_indices, "SHO"],
        "DEF": X_test.loc[attack_indices, "DEF"],
        "Real OVR": y_test.loc[attack_indices],
        "Predicted OVR": preds_attack
    })
    print("=== Angreifer-Szenario (hohe PAC & SHO) ===")
    display(df_attack)

# 2) Verteidiger-Szenario: Hohe DEF & niedriges SHO

def_mask = (X_test["DEF"] > 80) & (X_test["SHO"] < 50)
def_indices = X_test[def_mask].index

if len(def_indices) == 0:
    print("Keine Verteidiger mit (DEF>80 & SHO<50) im Testset!")
else:
   
    if len(def_indices) > 5:
        def_indices = np.random.choice(def_indices, 5, replace=False)

    preds_def = model.predict(X_test.loc[def_indices])
    df_def = pd.DataFrame({
        "PAC": X_test.loc[def_indices, "PAC"],
        "SHO": X_test.loc[def_indices, "SHO"],
        "DEF": X_test.loc[def_indices, "DEF"],
        "Real OVR": y_test.loc[def_indices],
        "Predicted OVR": preds_def
    })
    print("=== Verteidiger-Szenario (hohe DEF & niedriges SHO) ===")
    display(df_def)


=== Angreifer-Szenario (hohe PAC & SHO) ===


,PAC,SHO,DEF,Real OVR,Predicted OVR
8,90,84,52,89,87.96
88,82,82,57,85,83.08
16251,91,83,37,83,83.47
16214,85,82,41,85,84.07
0,97,90,36,91,89.28


=== Verteidiger-Szenario (hohe DEF & niedriges SHO) ===


,PAC,SHO,DEF,Real OVR,Predicted OVR
16455,72,33,82,79,79.41
373,65,45,82,80,79.66
137,67,32,85,83,83.81
200,60,37,83,82,82.16
16237,68,46,87,84,83.02


## 5) Manuelle Prüfung 

Um neben reinen Metriken (RMSE, R²) auch ein konkretes Gefühl für die Modellleistung zu bekommen, ziehe ich eine kleine Stichprobe echter Testinstanzen. Für jeden Spieler in dieser Stichprobe vergleiche ich die tatsächlichen Attribute (PAC, SHO, DEF etc.) sowie den realen OVR mit der Vorhersage. So sehe ich im Detail, ob das Modell erwartungsgemäss bei Stürmern (hohe PAC/SHO) oder Verteidigern (hohe DEF) einen passenden OVR schätzt. Diese Vorgehensweise schliesst die Lücke zwischen reinen Zahlen und der fußballpraktischen Sicht, indem ich kontrolliere, ob die Prognosen zu plausiblen Spielerprofilen passen.


In [12]:
# Manuelle Prüfung einer Zufallsstichprobe aus dem Testset
import numpy as np

# Wähle 5 zufällige Indizes aus dem Test-Set
sample_size = 5
sample_indices = np.random.choice(X_test.index, size=sample_size, replace=False)

# Vorhersagen des Modells für diese Stichprobe
predicted_ovr = model.predict(X_test.loc[sample_indices])

# Tabelle erstellen
manual_check_df = pd.DataFrame({
    "PAC": X_test.loc[sample_indices, "PAC"],
    "SHO": X_test.loc[sample_indices, "SHO"],
    "DEF": X_test.loc[sample_indices, "DEF"],
    "Real OVR": y_test.loc[sample_indices],
    "Predicted OVR": predicted_ovr
}, index=sample_indices)

print("=== Manuelle Stichprobe (5 zufällige Testinstanzen) ===")
display(manual_check_df.sort_index())


=== Manuelle Stichprobe (5 zufällige Testinstanzen) ===


,PAC,SHO,DEF,Real OVR,Predicted OVR
2589,74,73,31,73,73.67
5358,73,66,41,69,69.02
12817,89,61,28,61,62.54
13262,60,61,22,60,59.45
13719,78,30,51,60,58.39


## Manuelle Prüfung: 5 gezielte Profile 

Die bisherige Zufallsstichprobe liefert zwar einen groben Einblick, aber um wirklich fussballtypische Fälle zu testen, wähle ich nun gezielt fünf Szenarien aus: (1) einen Topstar mit sehr hohem OVR, (2) einen Spieler mit sehr niedrigem OVR, (3) einen Angreifer mit hohem PAC/SHO, (4) einen Verteidiger mit hohem DEF/niedrigem SHO und (5) einen balancierten Allrounder. Diese Auswahl deckt unterschiedliche Rollen und Leistungsniveaus ab, was eine realistischere Einschätzung ermöglicht. So wird erkennbar, ob das Modell für Ausreißer (ganz hoch/tief) und typische Profilwerte (z. B. starker Verteidiger) passend prognostiziert.


In [14]:
import numpy as np
import pandas as pd



# 1) Topstar: OVR > 85
topstar_indices = y_test[y_test > 85].index
topstar_pick = None
if len(topstar_indices) > 0:
    # Falls mehrere vorhanden, nehme eine zufällige
    topstar_pick = np.random.choice(topstar_indices, 1, replace=False)[0]

# 2) Niedriger OVR: <50
low_indices = y_test[y_test < 50].index
low_pick = None
if len(low_indices) > 0:
    low_pick = np.random.choice(low_indices, 1, replace=False)[0]

# 3) Angreifer: PAC>80 & SHO>80
attacker_mask = (X_test["PAC"] > 80) & (X_test["SHO"] > 80)
attacker_indices = X_test[attacker_mask].index
attacker_pick = None
if len(attacker_indices) > 0:
    attacker_pick = np.random.choice(attacker_indices, 1, replace=False)[0]

# 4) Verteidiger: DEF>80 & SHO<50
defender_mask = (X_test["DEF"] > 80) & (X_test["SHO"] < 50)
defender_indices = X_test[defender_mask].index
defender_pick = None
if len(defender_indices) > 0:
    defender_pick = np.random.choice(defender_indices, 1, replace=False)[0]

# 5) Allrounder: (Werte überall so um 60-70)

allrounder_mask = (
    (X_test["PAC"].between(60,70)) &
    (X_test["SHO"].between(60,70)) &
    (X_test["DEF"].between(60,70))
)
allrounder_indices = X_test[allrounder_mask].index
allrounder_pick = None
if len(allrounder_indices) > 0:
    allrounder_pick = np.random.choice(allrounder_indices, 1, replace=False)[0]

# Erstelle eine Liste mit den 5 Picks, falls sie existieren
picks = []
for pick in [topstar_pick, low_pick, attacker_pick, defender_pick, allrounder_pick]:
    if pick is not None:
        picks.append(pick)



if not picks:
    print("Keine gezielten Profile gefunden!")
else:
    picks = list(set(picks))  # Duplikate entfernen, falls sich Szenarien überschneiden

    # Vorhersagen
    preds = model.predict(X_test.loc[picks])

    # DataFrame bauen
    scenario_df = pd.DataFrame({
        "PAC": X_test.loc[picks, "PAC"],
        "SHO": X_test.loc[picks, "SHO"],
        "DEF": X_test.loc[picks, "DEF"],
        "Real OVR": y_test.loc[picks],
        "Predicted OVR": preds
    }, index=picks)
    scenario_df.sort_index(inplace=True)

    print("=== Manuelle Prüfung: 5 gezielte Profile ===")
    display(scenario_df)



=== Manuelle Prüfung: 5 gezielte Profile ===


,PAC,SHO,DEF,Real OVR,Predicted OVR
42,84,82,56,87,85.76
3411,66,69,65,71,69.69
16145,75,31,49,47,53.00
16237,68,46,87,84,83.02
16358,85,82,28,81,82.49


## 7) Schlussfazit 

Mein RandomForest-Modell sagt das OVR-Feld bereits mit hoher Präzision vorher, wie Metriken (RMSE ~1,2 und R² ~0,97) belegen. Der Train-/Test-Split beugt Overfitting vor, sodass die Vorhersagen auch auf unbekannte Daten sehr gut abschneiden. Besonders überzeugend sind die manuellen Prüfungen: Sowohl zufällig gewählte Testeinträge als auch gezielt ausgewählte Spielertypen (z. B. Angreifer mit hohem PAC/SHO, Verteidiger mit DEF>80) zeigen, dass das Modell die fussballtypische Rollenverteilung verstanden hat. Mit diesen Schritten habe ich ein solides, realitätsnahes Vorhersagemodell für EA Sports FC 25 erstellt.
